# Stages of cleaning the data used in the DAV hackathon

The data comes from the UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption

Source:

Georges HÃ©brail (georges.hebrail '@' edf.fr), Senior Researcher, EDF R&D, Clamart, France 
Alice BÃ©rard, TELECOM ParisTech Master of Engineering Internship at EDF R&D, Clamart, France


Data Set Information:

This archive contains 2075259 measurements gathered between December 2006 and November 2010 (47 months). 
Notes: 
1.(global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3) represents the active energy consumed every minute (in watt hour) in the household by electrical equipment not measured in sub-meterings 1, 2 and 3. 
2.The dataset contains some missing values in the measurements (nearly 1,25% of the rows). All calendar timestamps are present in the dataset but for some timestamps, the measurement values are missing: a missing value is represented by the absence of value between two consecutive semi-colon attribute separators. For instance, the dataset shows missing values on April 28, 2007.


Attribute Information:

1.date: Date in format dd/mm/yyyy 
2.time: time in format hh:mm:ss 
3.global_active_power: household global minute-averaged active power (in kilowatt) 
4.global_reactive_power: household global minute-averaged reactive power (in kilowatt) 
5.voltage: minute-averaged voltage (in volt) 
6.global_intensity: household global minute-averaged current intensity (in ampere) 
7.sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered). 
8.sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light. 
9.sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner.



For an example of other people looking at the data: http://thisisnic.github.io/EDA-of-Electric-Power-Consumption-Dataset/write_up.html 


### We'll take a subset of the data, remove missing values, format the date/time nicely, convert the columns to useable types and present it ready to be explored.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
%matplotlib inline

In [3]:
data_file = "household_power_consumption.txt"

In [4]:
data = pd.read_csv(data_file, sep=';') # We get a warning here - it can't guess column types since this dataset has missing values

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data.describe()

Sub_metering_3
count  2049280.000000
mean         6.458447
std          8.437154
min          0.000000
25%          0.000000
50%          1.000000
75%         17.000000
max         31.000000

Over 2 million records (133 MB). A little bit much, so we'll get a subsample.

In [6]:
df = data.iloc[20000:120000].copy() # .copy() added to avoid warnings when we modify the copy

In [7]:
df.columns

Index([u'Date', u'Time', u'Global_active_power', u'Global_reactive_power',
       u'Voltage', u'Global_intensity', u'Sub_metering_1', u'Sub_metering_2',
       u'Sub_metering_3'],
      dtype='object')

In [8]:
df.loc[df['Global_active_power'] == '?']

Date      Time Global_active_power Global_reactive_power Voltage  \
41832  14/1/2007  18:36:00                   ?                     ?       ?   
61909  28/1/2007  17:13:00                   ?                     ?       ?   
98254  22/2/2007  22:58:00                   ?                     ?       ?   
98255  22/2/2007  22:59:00                   ?                     ?       ?   

      Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
41832                ?              ?              ?             NaN  
61909                ?              ?              ?             NaN  
98254                ?              ?              ?             NaN  
98255                ?              ?              ?             NaN

Four rows with missing data. I cheat and replace this with the means, to avoid confusion

In [9]:
df.loc[df['Global_active_power'] == '?', 'Global_reactive_power'] = 0.128
df.loc[df['Global_active_power'] == '?', 'Voltage'] = 240.8
df.loc[df['Global_active_power'] == '?', 'Global_intensity'] = 6.96
df.loc[df['Global_active_power'] == '?', 'Sub_metering_1'] = 1.314
df.loc[df['Global_active_power'] == '?', 'Sub_metering_2'] = 1.9
df.loc[df['Global_active_power'] == '?', 'Sub_metering_3'] = 7.5
df.loc[df['Global_active_power'] == '?', 'Global_active_power'] = 1.64

In [10]:
df.loc[df['Global_reactive_power'] == '?'] #Those records have been fixed

Empty DataFrame
Columns: [Date, Time, Global_active_power, Global_reactive_power, Voltage, Global_intensity, Sub_metering_1, Sub_metering_2, Sub_metering_3]
Index: []

In [11]:
df.loc[df['Global_active_power'] == 1.64] # All fixed - with fake (mean) values

Date      Time Global_active_power Global_reactive_power Voltage  \
41832  14/1/2007  18:36:00                1.64                 0.128   240.8   
61909  28/1/2007  17:13:00                1.64                 0.128   240.8   
98254  22/2/2007  22:58:00                1.64                 0.128   240.8   
98255  22/2/2007  22:59:00                1.64                 0.128   240.8   

      Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
41832             6.96          1.314            1.9             7.5  
61909             6.96          1.314            1.9             7.5  
98254             6.96          1.314            1.9             7.5  
98255             6.96          1.314            1.9             7.5

In [12]:
# df = df.loc[df['Global_active_power']!="?"] # Removing missing records - another option

In [13]:
df[df.columns[2:]] = df[df.columns[2:]].astype(float) # convert all the non-date columns to floats

In [14]:
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True, errors='ignore') #takes a while

In [15]:
df.head()

Date      Time  Global_active_power  Global_reactive_power  \
20000  30/12/2006  14:44:00                2.014                  0.402   
20001  30/12/2006  14:45:00                2.170                  0.512   
20002  30/12/2006  14:46:00                2.194                  0.520   
20003  30/12/2006  14:47:00                2.172                  0.514   
20004  30/12/2006  14:48:00                2.188                  0.518   

       Voltage  Global_intensity  Sub_metering_1  Sub_metering_2  \
20000   240.33               8.6               0               1   
20001   240.96               9.2               0               0   
20002   241.18               9.2               0               0   
20003   240.98               9.2               0               0   
20004   241.02               9.2               0               0   

       Sub_metering_3            Datetime  
20000              18 2006-12-30 14:44:00  
20001              18 2006-12-30 14:45:00  
20002              17 2006-12-30 14:46:00  
20003              18 2006-12-30 14:47:00  
20004              18 2006-12-30 14:48:00

At this point, we have a nice clean dataset for the hackathon

In [16]:
df.to_pickle('power_consumption')